In [1]:
import numpy as np
import pyedflib
import statistics
import plotly.graph_objects as go
import pandas as pd
from gtda.time_series import SingleTakensEmbedding
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PersistenceEntropy, Amplitude, NumberOfPoints, ComplexPolynomial, PersistenceLandscape, HeatKernel, Silhouette, BettiCurve, PairwiseDistance, ForgetDimension
from gtda.plotting import plot_point_cloud, plot_heatmap, plot_diagram
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA, FastICA
from gtda.pipeline import Pipeline 


def read_edf_file(file_path):
    """
    Reads an .edf file and returns the EEG and EMG streams as pandas DataFrames.
    """
    f = pyedflib.EdfReader(file_path)

    # Assuming the EEG channel is the first channel and EMG is the second channel
    eeg_signal = f.readSignal(0)
    emg_signal = f.readSignal(1)

    # Extract the channel names for the DataFrame
    eeg_channel_name = f.getSignalLabels()[0]
    emg_channel_name = f.getSignalLabels()[1]

    # Get the sample frequency
    sample_frequency = f.getSampleFrequency(0)  # Assuming both streams have the same frequency

    # Calculate the timestamps for the samples
    n_samples = min(len(eeg_signal), len(emg_signal))
    time = [i / sample_frequency for i in range(n_samples)]

    # Create pandas DataFrame
    df = pd.DataFrame({
        'Time': time,
        eeg_channel_name: eeg_signal[:n_samples],
        emg_channel_name: emg_signal[:n_samples],
    })

    # Close the EdfReader
    f.close()

    return df

file = 'edf_293.edf'

data = read_edf_file(file)


x = data.Time
y = data.EEG

In [2]:
# Labels
label_df = pd.read_csv("Data_293.csv")
labels = label_df["NAPS_Numeric"].iloc[1:]
labels = [int(label) for label in labels]

# Label List

Label 1: W (Awake)

Label 2: WA (Awake Artifact)?

Label 3: NR (NREM)

Label 4: Not defined

Label 5: R (REM)

Label 7: U (Artifacts?)


# Local Approach

In [3]:
# How many segments per label do you want to analyze?
no_segments = 100

In [4]:
indices_dict = {}

for label in list(set(labels)): 
    indices = [index for index, value in enumerate(labels) if value == label][:no_segments]
    indices_dict[label] = indices

In [5]:
def segment_data(df, segment_size, step_size = 2):
    n_segments = int(df["Time"].iloc[-1]) // segment_size
    eeg_segments = []
    emg_segments = []

    for i in range(n_segments):
        start_idx = int(i* segment_size*1000/step_size)
        end_idx = start_idx + int(segment_size*1000/step_size)
        segment = df.iloc[start_idx:end_idx]
        eeg_segments.append(list(segment["EEG"]))
        emg_segments.append(list(segment["EMG"]))

    return eeg_segments, emg_segments

In [6]:
# Segment the data
segment_length = 4  # seconds
eeg_segments, emg_segments = segment_data(data, segment_length, step_size = 2)

## Finding the optimal embedding dimension and time delay

There are two techniques that can be used to determine these parameters automatically:
- Mutual information to determine the time delay
- False nearest neighbours to determine the embedding dimension

In [7]:
# Initialise the embedding

max_embedding_dimension = 30
max_time_delay = 30
stride = 5

embedder = SingleTakensEmbedding(
    parameters_type="search",
    time_delay=max_time_delay,
    dimension=max_embedding_dimension,
    stride=stride,
)

In [8]:
def fit_embedder(embedder: SingleTakensEmbedding, y: np.ndarray, verbose: bool=True) -> np.ndarray:
    """Fits a Takens embedder and displays optimal search parameters."""
    y_embedded = embedder.fit_transform(y)

    if verbose:
        print(f"Shape of embedded time series: {y_embedded.shape}")
        print(
            f"Optimal embedding dimension is {embedder.dimension_} and time delay is {embedder.time_delay_}"
        )

    return y_embedded

In [9]:
# Look at some random segments
y_embedded = fit_embedder(embedder, eeg_segments[0])
y_embedded = fit_embedder(embedder, eeg_segments[100])
y_embedded = fit_embedder(embedder, eeg_segments[177])
y_embedded = fit_embedder(embedder, eeg_segments[1000])
# The optimal values are all similar (=> Just use embedding dimension 5 and time delay 25)

Shape of embedded time series: (380, 5)
Optimal embedding dimension is 5 and time delay is 26
Shape of embedded time series: (383, 4)
Optimal embedding dimension is 4 and time delay is 29
Shape of embedded time series: (383, 5)
Optimal embedding dimension is 5 and time delay is 22
Shape of embedded time series: (373, 6)
Optimal embedding dimension is 6 and time delay is 27


## Creating Persistence Diagrams

In [10]:
# Setting parameters for point cloud embeddings

embedding_dimension= 5
embedding_time_delay = 25
stride = 10

embedder_periodic = SingleTakensEmbedding(
    parameters_type="fixed",
    n_jobs=2,
    time_delay=embedding_time_delay,
    dimension=embedding_dimension,
    stride=stride,
)

In [11]:
# We will look at 0, 1 and 2 dimensional holes TODO try more?
homology_dimensions = [0, 1, 2]

# We will use a Vietoris Rips filtrations
persistence = VietorisRipsPersistence(
    homology_dimensions=homology_dimensions, n_jobs=10
)

### Computing Points Clouds and Persistence Diagrams

In [12]:
# Label 1

# Point cloud embeddings
y_embedded1 = {} 

# Persistence diagrams
diagrams1 = {}

# Loop through the first segments with label '1'
for label_idx in indices_dict[1]:
    y_embedded1[label_idx] = embedder_periodic.fit_transform(eeg_segments[label_idx])[None, :, :]
    diagrams1[label_idx] = persistence.fit_transform(y_embedded1[label_idx])

In [14]:
# Label 3

# Point cloud embeddings
y_embedded3 = {} 

# Persistence diagrams
diagrams3 = {}

# Loop through the first segments with label '3'
for label_idx in indices_dict[3]:
    y_embedded3[label_idx] = embedder_periodic.fit_transform(eeg_segments[label_idx])[None, :, :]
    diagrams3[label_idx] = persistence.fit_transform(y_embedded3[label_idx])

In [15]:
# Label 5

# Point cloud embeddings
y_embedded5 = {} 

# Persistence diagrams
diagrams5 = {}

# Loop through the first segments with label '5'
for label_idx in indices_dict[5]:
    y_embedded5[label_idx] = embedder_periodic.fit_transform(eeg_segments[label_idx])[None, :, :]
    diagrams5[label_idx] = persistence.fit_transform(y_embedded5[label_idx])

In [16]:
# Label 7

# Point cloud embeddings
y_embedded7 = {} 

# Persistence diagrams
diagrams7 = {}

# Loop through the first segments with label '1'
for label_idx in indices_dict[7]:
    y_embedded7[label_idx] = embedder_periodic.fit_transform(eeg_segments[label_idx])[None, :, :]
    diagrams7[label_idx] = persistence.fit_transform(y_embedded7[label_idx])

## Summary Statistics

In [24]:
# These are the features that we will examine for the persistence diagrams of each label
PE = PersistenceEntropy()
AM = Amplitude()
NP = NumberOfPoints()
CP = ComplexPolynomial()

In [25]:
persistence_entropy1 = {}
amplitude1 = {}
no_points1 = {}
complex_polynomial1 = {}

for label_idx in indices_dict[1]:
    persistence_entropy1[label_idx] = PE.fit_transform(diagrams1[label_idx])
    amplitude1[label_idx] = AM.fit_transform(diagrams1[label_idx])
    no_points1[label_idx] = NP.fit_transform(diagrams1[label_idx])
    complex_polynomial1[label_idx] = CP.fit_transform(diagrams1[label_idx])

In [26]:
persistence_entropy3 = {}
amplitude3 = {}
no_points3 = {}
complex_polynomial3 = {}


for label_idx in indices_dict[3]:
    persistence_entropy3[label_idx] = PE.fit_transform(diagrams3[label_idx])
    amplitude3[label_idx] = AM.fit_transform(diagrams3[label_idx])
    no_points3[label_idx] = NP.fit_transform(diagrams3[label_idx])
    complex_polynomial3[label_idx] = CP.fit_transform(diagrams3[label_idx])

In [27]:
persistence_entropy5 = {}
amplitude5 = {}
no_points5 = {}
complex_polynomial5 = {}


for label_idx in indices_dict[5]:
    persistence_entropy5[label_idx] = PE.fit_transform(diagrams5[label_idx])
    amplitude5[label_idx] = AM.fit_transform(diagrams5[label_idx])
    no_points5[label_idx] = NP.fit_transform(diagrams5[label_idx])
    complex_polynomial5[label_idx] = CP.fit_transform(diagrams5[label_idx])

In [28]:
persistence_entropy7 = {}
amplitude7 = {}
no_points7 = {}
complex_polynomial7 = {}


for label_idx in indices_dict[7]:
    persistence_entropy7[label_idx] = PE.fit_transform(diagrams7[label_idx])
    amplitude7[label_idx] = AM.fit_transform(diagrams7[label_idx])
    no_points7[label_idx] = NP.fit_transform(diagrams7[label_idx])
    complex_polynomial7[label_idx] = CP.fit_transform(diagrams7[label_idx])

# Removing noise

In [33]:
def cut_diagrams(flattened_diagrams, no_holes_per_dimension):
    # no_holes_per_dimension is a list indicating how many holes for each dimension there should be left
    shortened_diagrams = []

    for diagram in flattened_diagrams: # There are no_segments many diagrams per label at max (as chosen in the beginning)
        most_significant_holes_per_diagram = []
        holes = {}
        for hole_dimension, number_of_holes in zip(range(3), no_holes_per_dimension):
            # the third entry of each point (hole) in a diagram indicates its dimensionality
            holes[hole_dimension] = diagram[np.where(diagram[:, 2] == hole_dimension)[0]]

            if number_of_holes > len(holes[hole_dimension]):
                print("Watch out! There is a diagram shorter than the shortened diagrams")
                print("It has " + str(len(holes[hole_dimension])) + " holes of dimension " + str(hole_dimension))

            # The first and second entries of each hole indicate its birth and death, the difference is the persistence
            large_persistence_indices = np.argsort(holes[hole_dimension][:, 0] - holes[hole_dimension][:, 1])[-number_of_holes:]
            
            # For each dimension, getting the holes with the above indices (the holes with the largest persistence)
            significant_holes_with_hole_dimension = holes[hole_dimension][large_persistence_indices, :]
            most_significant_holes_per_diagram.extend(significant_holes_with_hole_dimension)

        shortened_diagrams.append(most_significant_holes_per_diagram)

    return shortened_diagrams

In [34]:
# Label 1

no_holes_per_dimension1 = [110, 80, 15]

flattened_diagrams = []
for key, value in diagrams1.items():
    flattened_diagrams.append(value[0])

shortened_diagrams1 = cut_diagrams(flattened_diagrams, no_holes_per_dimension1)

In [35]:
# Label 3

no_holes_per_dimension3 = [100, 68, 7]

flattened_diagrams = []
for key, value in diagrams3.items():
    flattened_diagrams.append(value[0])

shortened_diagrams3 = cut_diagrams(flattened_diagrams, no_holes_per_dimension3)

In [36]:
# Label 5

no_holes_per_dimension5 = [120, 60, 12] 

flattened_diagrams = []
for key, value in diagrams5.items():
    flattened_diagrams.append(value[0])

shortened_diagrams5 = cut_diagrams(flattened_diagrams, no_holes_per_dimension5)

In [37]:
# Label 7

# One "outlier" diagram only has one 2-dimensional hole
# Later, such outliers should be deleted before computing the pairwise distances
# between all diagrams, because eventually all diagrams should be shortened to the
# length of the shortest diagram

no_holes_per_dimension7 = [50, 9, 1] 


flattened_diagrams = []
for key, value in diagrams7.items():
    flattened_diagrams.append(value[0])

shortened_diagrams7 = cut_diagrams(flattened_diagrams, no_holes_per_dimension7)

# Signatures and their Statistics

## HeatKernel

In a way, the Heat Kernel shows an "average distribution" of the persistence diagrams for each label, seperated per hole dimensionality.

In [44]:
HK = HeatKernel(sigma=0.00003, n_bins=100)


### Computing average intensity of the classes

Does this value stay the same if I average the intensity of the single heat kernels for each PD?

Label 1

In [45]:
heatkernel = HK.fit_transform(shortened_diagrams1)

In [47]:
average_intensity1 = np.mean(heatkernel)
print(average_intensity1)

6.200472513834635e-10


Label 3

In [54]:
heatkernel = HK.fit_transform(shortened_diagrams3)

In [55]:
average_intensity3 = np.mean(heatkernel)
print(average_intensity3)

4.141807556152343e-09


Label 5

In [56]:
heatkernel = HK.fit_transform(shortened_diagrams5)

average_intensity5 = np.mean(heatkernel)
print(average_intensity5)

-7.346824363425926e-10


There is something wrong with this being negative

### Computing average intensity of the single PDs' heat kernels as features


In [51]:
heatkernel = HK.fit_transform([shortened_diagrams3[0]])

## Persistance Landscape

In [58]:
PL = PersistenceLandscape()

Label 1

Persistence landscapes map persistence diagrams into a function space, which may often be taken to be a Banach space or even a Hilbert space

In [83]:
persistence_landscape1 = PL.fit_transform(shortened_diagrams1)

Label 3

In [64]:
persistence_landscape3 = PL.fit_transform(shortened_diagrams3)

Label 5

In [65]:
persistence_landscap5e = PL.fit_transform(shortened_diagrams5)

### Computing the L1 norm of the persistence landscapes for different dimensions

## Silhouette

In [67]:
SH = Silhouette()

Label 1

In [79]:
silhouette1 = SH.fit_transform(shortened_diagrams1)

Label 3

In [80]:
silhouette3 = SH.fit_transform(shortened_diagrams3)

Label 5

In [81]:
silhouette5 = SH.fit_transform(shortened_diagrams5)

# Concatenate Features to one DataFrame

In [107]:
feature_df = pd.DataFrame(columns = ["Label", "Amplitude_Dim_0", "Amplitude_Dim_1", "Amplitude_Dim_2",  
                                     "Persistence Entropy_Dim_0",  "Persistence Entropy_Dim_1",  "Persistence Entropy_Dim_2", 
                                     "No_Points_Dim_0", "No_Points_Dim_1", "No_Points_Dim_2"])

In [127]:
def column(matrix, i):
    return [row[0][i] for row in matrix]

In [129]:
feature_df1 = pd.DataFrame()

feature_df1["Persistence Entropy_Dim_0"] = column(list(persistence_entropy1.values()), 0)
feature_df1["Persistence Entropy_Dim_1"] = column(list(persistence_entropy1.values()), 1)
feature_df1["Persistence Entropy_Dim_2"] = column(list(persistence_entropy1.values()), 2)
feature_df1["Amplitude_Dim_0"] = column(list(amplitude1.values()), 0)
feature_df1["Amplitude_Dim_1"] = column(list(amplitude1.values()), 1)
feature_df1["Amplitude_Dim_2"] = column(list(amplitude1.values()), 2)
feature_df1["No_Points_Dim_0"] = column(list(no_points1.values()), 0)
feature_df1["No_Points_Dim_1"] = column(list(no_points1.values()), 1)
feature_df1["No_Points_Dim_2"] = column(list(no_points1.values()), 2)
feature_df1["Label"] = 1

In [131]:
feature_df3 = pd.DataFrame()

feature_df3["Persistence Entropy_Dim_0"] = column(list(persistence_entropy3.values()), 0)
feature_df3["Persistence Entropy_Dim_1"] = column(list(persistence_entropy3.values()), 1)
feature_df3["Persistence Entropy_Dim_2"] = column(list(persistence_entropy3.values()), 2)
feature_df3["Amplitude_Dim_0"] = column(list(amplitude1.values()), 0)
feature_df3["Amplitude_Dim_1"] = column(list(amplitude1.values()), 1)
feature_df3["Amplitude_Dim_2"] = column(list(amplitude1.values()), 2)
feature_df3["No_Points_Dim_0"] = column(list(no_points1.values()), 0)
feature_df3["No_Points_Dim_1"] = column(list(no_points1.values()), 1)
feature_df3["No_Points_Dim_2"] = column(list(no_points1.values()), 2)
feature_df3["Label"] = 3

In [132]:
feature_df = pd.concat([feature_df1, feature_df3])

In [133]:
feature_df.to_csv("Features.csv")